# GBDT training on RTX 3090 (GPU-accelerated)

This notebook trains Gradient Boosting Decision Trees on `dataset_pruned.csv` using GPU acceleration where available:

- XGBoost (gpu_hist)
- LightGBM (device='gpu', fallback to CPU if GPU build is unavailable)

It follows the paper's setup (see `#2.3_GBDT.txt`): 70/30 split, normalization, paper hyperparameters, and evaluation with MAE/MSE/RMSE/R2.


In [19]:
import os
import json
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

DATA_PATH = 'dataset_pruned.csv'
LABEL_COL = 'weight_kg'
ID_COLS = ['Chicken_ID', 'Image_ID']

assert os.path.exists(DATA_PATH), f"Missing {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
print(f"Loaded: {df.shape[0]} rows, {df.shape[1]} cols")

y = df[LABEL_COL].values
X = df.drop(columns=ID_COLS + [LABEL_COL])
feature_names = X.columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, shuffle=True
)

scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Train/Test:", X_train_scaled.shape, X_test_scaled.shape)


Loaded: 1492 rows, 2076 cols
Train/Test: (1044, 2073) (448, 2073)


In [20]:
# Data cleaning and imputation to avoid NaNs/inf and dtype issues
import numpy as np
from sklearn.impute import SimpleImputer

# Drop rows with missing label
before_rows = df.shape[0]
df = df[~df[LABEL_COL].isna()].reset_index(drop=True)
print(f"Dropped {before_rows - df.shape[0]} rows with NaN in label {LABEL_COL}")

# Coerce all feature columns to numeric and replace inf with NaN
feature_cols = [c for c in df.columns if c not in (ID_COLS + [LABEL_COL])]
for c in feature_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Replace +/-inf with NaN, then we will impute
df[feature_cols] = df[feature_cols].replace([np.inf, -np.inf], np.nan)

# Recompute X, y after cleaning
y = df[LABEL_COL].values
X = df[feature_cols]

# Train/test split 70/30
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, shuffle=True
)

# Impute missing numeric features using training medians (no leakage), then scale
imputer = SimpleImputer(strategy='median')
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train_imp)
X_test_scaled = scaler.transform(X_test_imp)

print("After cleaning/imputation:", X_train_scaled.shape, X_test_scaled.shape)



Dropped 1 rows with NaN in label weight_kg
After cleaning/imputation: (1043, 2073) (448, 2073)


In [21]:
# pip install lightgbm xgboost numpy pandas scikit-learn

In [22]:
# XGBoost GPU config (RTX 3090)
from xgboost import XGBRegressor

xgb_params_gpu = dict(
    n_estimators=2000,
    learning_rate=0.1,
    max_depth=5,
    objective='reg:squarederror',
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.8,
    colsample_bytree=0.7,
    colsample_bylevel=1.0,
    reg_alpha=0.0,
    reg_lambda=1.0,
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    gpu_id=0,
    random_state=42,
    n_jobs=0,
)

xgb_gpu = XGBRegressor(**xgb_params_gpu)
xgb_gpu.fit(X_train_scaled, y_train)
print("XGB (GPU) trained.")


XGB (GPU) trained.


In [23]:
# LightGBM GPU config (fall back to CPU if GPU build not present)
import lightgbm as lgb

def build_lgb_gpu():
    params = dict(
        n_estimators=4000,
        learning_rate=0.1,
        num_leaves=15,
        max_depth=5,
        min_child_samples=15,
        min_child_weight=0.01,
        subsample=0.8,
        colsample_bytree=1.0,
        objective='regression',
        n_jobs=-1,
        device='gpu',
        gpu_platform_id=0,
        gpu_device_id=0,
    )
    try:
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train_scaled, y_train)
        return model, True
    except Exception as e:
        print("LightGBM GPU failed, falling back to CPU:", e)
        params.pop('device', None)
        params.pop('gpu_platform_id', None)
        params.pop('gpu_device_id', None)
        model = lgb.LGBMRegressor(**params)
        model.fit(X_train_scaled, y_train)
        return model, False

lgb_model, lgb_used_gpu = build_lgb_gpu()
print(f"LGBM trained. GPU used: {lgb_used_gpu}")


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 528285
[LightGBM] [Info] Number of data points in the train set: 1043, number of used features: 2073
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3090, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 2073 dense feature groups (2.06 MB) transferred to GPU in 0.020305 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 1.337254
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

In [24]:
# Save imputer separately
import os, joblib
os.makedirs('artifacts_3090', exist_ok=True)
joblib.dump(imputer, 'artifacts_3090/imputer.joblib')
print('Saved imputer to artifacts_3090/imputer.joblib')


Saved imputer to artifacts_3090/imputer.joblib


In [25]:
# Silence LightGBM info/warnings by setting verbosity
import warnings as _warnings
_warnings.filterwarnings("ignore", category=UserWarning, module="lightgbm")

import lightgbm as lgb
_lgb_params = lgb_model.get_params()
_lgb_params["verbosity"] = -1
_lgb_params["verbose"] = -1

lgb_model = lgb.LGBMRegressor(**_lgb_params)
lgb_model.fit(X_train_scaled, y_train)
print("Re-trained LightGBM with verbosity=-1 (warnings suppressed)")


KeyboardInterrupt: 

In [ ]:
# Save models and training details to "saved models"
import os, json, joblib, datetime
save_dir = 'saved models'
os.makedirs(save_dir, exist_ok=True)

# Save artifacts
joblib.dump(imputer, os.path.join(save_dir, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(save_dir, 'scaler.joblib'))
joblib.dump(lgb_model, os.path.join(save_dir, 'lgbm_model.joblib'))
joblib.dump(xgb_gpu, os.path.join(save_dir, 'xgb_gpu_model.joblib'))

# Save metrics
with open(os.path.join(save_dir, 'metrics.json'), 'w') as f:
    json.dump(results, f, indent=2)

# Save a human-readable training summary
details = []
details.append(f"Timestamp: {datetime.datetime.now().isoformat()}")
details.append(f"Dataset: dataset_pruned.csv")
details.append(f"Train/Test shapes: {X_train_scaled.shape} / {X_test_scaled.shape}")

# LightGBM params
try:
    lgb_params_dump = lgb_model.get_params()
except Exception:
    lgb_params_dump = {}

# XGB params
try:
    xgb_params_dump = xgb_gpu.get_params()
except Exception:
    xgb_params_dump = {}

summary = {
    'lgbm_used_gpu': bool(lgb_used_gpu),
    'lgbm_params': lgb_params_dump,
    'xgb_params': xgb_params_dump,
    'results': results,
}

with open(os.path.join(save_dir, 'training_3090_details.txt'), 'w') as f:
    f.write('\n'.join(details) + '\n')
    f.write(json.dumps(summary, indent=2))

print(f"Saved models and details to: {save_dir}")


In [ ]:
def evaluate(model, X_tr, y_tr, X_te, y_te, name):
    pred_te = model.predict(X_te)
    mae = mean_absolute_error(y_te, pred_te)
    mse = mean_squared_error(y_te, pred_te)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_te, pred_te)
    print(f"{name} -> MAE: {mae:.6f}  MSE: {mse:.6f}  RMSE: {rmse:.6f}  R2: {r2:.6f}")
    return dict(model=name, MAE=mae, MSE=mse, RMSE=rmse, R2=r2)

results = []
results.append(evaluate(lgb_model, X_train_scaled, y_train, X_test_scaled, y_test, f"LGBM_GPU={lgb_used_gpu}"))
results.append(evaluate(xgb_gpu, X_train_scaled, y_train, X_test_scaled, y_test, 'XGB_GPU'))

pd.DataFrame(results)


LGBM_GPU=True -> MAE: 0.068463  MSE: 0.009457  RMSE: 0.097249  R2: 0.870522
XGB_GPU -> MAE: 0.069832  MSE: 0.010160  RMSE: 0.100797  R2: 0.860904


,model,MAE,MSE,RMSE,R2
0,LGBM_GPU=True,0.068463,0.009457,0.097249,0.870522
1,XGB_GPU,0.069832,0.010160,0.100797,0.860904


In [ ]:
# Save artifacts
os.makedirs('artifacts_3090', exist_ok=True)
joblib.dump(scaler, 'artifacts_3090/scaler.joblib')
joblib.dump(lgb_model, 'artifacts_3090/lgbm_model.joblib')
joblib.dump(xgb_gpu, 'artifacts_3090/xgb_gpu_model.joblib')

with open('artifacts_3090/metrics.json', 'w') as f:
    json.dump(results, f, indent=2)
print("Saved GPU run artifacts to artifacts_3090/")


Saved GPU run artifacts to artifacts_3090/
